In [0]:
%python
## Import libraries
import pyspark as ps
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import *
import pandas as pd
import json

In [0]:
# Retrieve the parameter values
job_id = dbutils.widgets.get("job_id")
usecase_id = dbutils.widgets.get("usecase_id")

# Print the parameter values
print(f"job_id: {job_id}")
print(f"usecase_id: {usecase_id}")

In [0]:
class GetSuggestion:
  def __init__(self):
    self.query = "Query About to fill"
    self.sql_df = pd.DataFrame()
    self.data_json = {"Data":[]}    
    self.usecase_id = None
  
  def logger(self, message):
    print(message)
    
  def set_query_string(self, usecase_parm):
    query = f"""WITH t(usecase_id, NGEBOX_RECORD_ID, zip_cd, id, account_vod__c) AS 
(select distinct request_table.usecase_id, request_table.NGEBOX_RECORD_ID, terr.zip_cd, usr.id, request_table.account_vod__c from com_us_alyt_ngebox.ib_NGEBox_Suggestion_Requests  request_table 
left join com_us_alyt_ngebox.METADATA_USECASE met on request_table.usecase_id = met.USECASE_ID
join com_us_alyt_ngebox.lkp_prod_salesforce  lkup on lkup.Prod_Nm = met.Prod_Nm
left join (select distinct sf_cd,rpt_sales_org_key, zip_cd, geo_cd from com_us_hub.ref_zip_terr
  where curr_ind = 'Y' and level = 'Territory' ) terr
on request_table.hcp_zip_code = terr.zip_cd
and terr.sf_cd = lkup.sf_cd
left join com_us_hub.ref_emp_assignment prime_assign
on terr.geo_cd = prime_assign.geo_code and prime_assign.assignment_typ = 'Primary'
and current_date() between prime_assign.assignment_start_date and prime_assign.assignment_end_date
left join com_us_hub.ref_emp_assignment secondary_assign
on terr.geo_cd = secondary_assign.geo_code and secondary_assign.assignment_typ = 'Secondary'
and current_date() between secondary_assign.assignment_start_date and secondary_assign.assignment_end_date
inner join com_us_hub.ref_iengage_user usr
on coalesce(prime_assign.emp_id,secondary_assign.emp_id) = usr.EMPLOYEENUMBER and usr.isactive = 1
where request_table.usecase_id = '{usecase_parm}')

merge into com_us_alyt_ngebox.NGEBox_Suggestion_Requests_Processed process_table 
using (select distinct tb1.usecase_id ,tb2.id,tb1.NGEBOX_RECORD_ID as NB_record_processed,tb2.NGEBOX_RECORD_ID as NB_record_t, tb4.hist_retention,tb1.suggest_external_id_vod__c  from com_us_alyt_ngebox.NGEBox_Suggestion_Requests_Processed tb1
inner join t tb2 
on tb1.NGEBOX_RECORD_ID = tb2.NGEBOX_RECORD_ID  and tb1.usecase_id = tb2.usecase_id
left join com_us_lake.rep_suggestion_vod__c tb3 
on tb3.SUGGESTION_EXTERNAL_ID_VOD__C = tb1.suggest_external_id_vod__c
inner join com_us_alyt_ngebox.metadata_usecase tb4 
on tb1.usecase_id = tb4.usecase_id
where tb1.usecase_id = '{usecase_parm}' and (tb3.SUGGESTION_EXTERNAL_ID_VOD__C is NULL) 
or (tb3.SUGGESTION_EXTERNAL_ID_VOD__C is not null and tb3.DISMISSED_VOD__C = 0)) source_table 
on (process_table.usecase_id =- source_table.usecase_id and process_table.NGEBOX_RECORD_ID = source_table.NB_record_processed and process_table.NGEBOX_RECORD_ID = source_table.NB_record_t and process_table.suggest_external_id_vod__c = source_table.suggest_external_id_vod__c 
and (case when process_table.usecase_id = '512'
 then (date(process_table.created_date) <> current_date and  date(process_table.created_date) >= date_add(current_date,-source_table.hist_retention))
else (date(process_table.created_date) >= date_add(current_date,-source_table.hist_retention))end))
when matched then
update set process_table.ownerid = source_table.id;"""
    self.query = query

  def run_query_and_set_sqldf(self, usecase_id):
      self.usecase_id = usecase_id
      query_list = self.query.split(";")
      self.sql_df = [] 
      for curr_query in query_list:
          if curr_query.strip() == "":
              continue
          curr_query_with_param = curr_query.replace("Usecase_parm", usecase_id)
          sql_df = sqlContext.sql(curr_query_with_param).collect()
          self.sql_df.extend(sql_df)
    
  def format_sqldf_to_json(self):
    sql_output_rowlist = []
    for curr_row in self.sql_df: 
      sql_output_rowlist.append (curr_row.asDict(True))
    sql_df_new = pd.DataFrame(sql_output_rowlist)
    sql_jsonstr = sql_df_new.to_json(orient="records") # type(result) -> str
    sql_jsonparse = json.loads(sql_jsonstr) # type(parsed_json) -> list
    nb_json = {"data":sql_jsonparse} # type(metadata_json) -> dict
    self.nb_json = nb_json
    
  def construct_microservice_response(self):
    resp = {
      "nge_response": {
        "status": 200,
        "body": self.nb_json
      }
    }
    self.response = resp

In [0]:
usecase_id = dbutils.widgets.get("usecase_id")
nb_obj = GetSuggestion()
nb_obj.set_query_string(usecase_parm=usecase_id)
nb_obj.run_query_and_set_sqldf(usecase_id=usecase_id)
nb_obj.format_sqldf_to_json()
nb_obj.construct_microservice_response()
dbutils.notebook.exit(nb_obj.response)

{'nge_response': {'status': 200, 'body': {'data': [{'num_affected_rows': 138, 'num_updated_rows': 138, 'num_deleted_rows': 0, 'num_inserted_rows': 0}]}}}